# Database

In [ ]:
from face import FaceRecognizer, FaceDatabase

In [ ]:
photos_path = 'data/photos'
embeddings_path = 'data/embeddings'
face_recognizer = FaceRecognizer()
face_database = FaceDatabase(embeddings_path)
face_database.add(photos_path, face_recognizer)

# How to use (example for 2 frames)

In [ ]:
import cv2
from face import FaceRecognizer, FaceDatabase

In [ ]:
video_path = 'data/example.mp4'
embeddings_path = 'data/embeddings'

In [ ]:
face_recognizer = FaceRecognizer()
face_database = FaceDatabase(embeddings_path)
face_recognizer.upload_database(face_database)

In [ ]:
cap = cv2.VideoCapture(video_path)
if cap.isOpened():
    ret, first_frame = cap.read()
    ret, second_frame = cap.read()
cap.release()

### Option 1. Detector + Encoder -> Detector + Encoder -> Detector + Encoder -> ...

In [ ]:
# first frame
bboxes, landmarks, confs = face_recognizer.detect(first_frame)
embeddings = face_recognizer.encode(first_frame, landmarks)
names = face_recognizer.identify(embeddings)

In [ ]:
# second frame
bboxes, landmarks, confs = face_recognizer.detect(second_frame)
embeddings = face_recognizer.encode(second_frame, landmarks)
names = face_recognizer.identify(embeddings)

### Option 2. Detector + Encoder -> OpticalFlowTracker -> OpticalFlowTracker -> ...

In [ ]:
from face import OpticalFlowTracker

In [ ]:
# first frame
bboxes, landmarks, confs = face_recognizer.detect(first_frame)
embeddings = face_recognizer.encode(first_frame, landmarks)
names = face_recognizer.identify(embeddings)
oft = OpticalFlowTracker(first_frame, bboxes, names)

In [ ]:
# second frame
ret, bboxes, names = oft.track(second_frame)

### Option 3. Detector + Encoder -> Detector + KalmanFilterTracker -> Detector + KalmanFilterTracker -> ...

In [ ]:
from face import KalmanFilterTracker

In [ ]:
# first frame
bboxes, landmarks, confs = face_recognizer.detect(first_frame)
embeddings = face_recognizer.encode(first_frame, landmarks)
names = face_recognizer.identify(embeddings)
kft = KalmanFilterTracker(bboxes, confs, names)

In [ ]:
# second frame
bboxes, _, confs = face_recognizer.detect(second_frame)
ret, bboxes, names = kft.track(bboxes, confs)

# How to use (full video example)

In [ ]:
import cv2
from face import FaceRecognizer, FaceDatabase, OpticalFlowTracker, KalmanFilterTracker

In [ ]:
mode = 'detector_encoder'
# mode = 'optical_flow_tracking'
# mode = 'tracking_by_detection'

In [ ]:
video_path = 'data/example.mp4'
embeddings_path = 'data/embeddings'

In [ ]:
face_recognizer = FaceRecognizer()
face_database = FaceDatabase(embeddings_path)
face_recognizer.upload_database(face_database)

In [ ]:
cap = cv2.VideoCapture(video_path)
output_video_path = ''.join(video_path.split('.')[:-1]) + '_processed' + '.mp4'
video_width, video_height, video_fps = int(cap.get(3)), int(cap.get(4)), int(cap.get(5))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, video_fps, (video_width, video_height))
initialize = True
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if not initialize:
        if mode == 'optical_flow_tracking':
            ret, bboxes, names = oft.track(frame)
            if not ret:
                initialize = True
                print('OpticalFlowTracker cannot locate one of the targets in current frame')
        elif mode == 'tracking_by_detection':
            bboxes, _, confs = face_recognizer.detect(frame)
            ret, bboxes, names = kft.track(bboxes, confs)
            if not ret:
                initialize = True
                print('KalmanFilterTracker has located new unknown target')
        elif mode == 'detector_encoder':
            bboxes, landmarks, confs = face_recognizer.detect(frame)
            embeddings = face_recognizer.encode(frame, landmarks)
            names = face_recognizer.identify(embeddings)
    if initialize:
        bboxes, landmarks, confs = face_recognizer.detect(frame)
        embeddings = face_recognizer.encode(frame, landmarks)
        names = face_recognizer.identify(embeddings)
        if mode == 'optical_flow_tracking':
            oft = OpticalFlowTracker(frame, bboxes, names)
        elif mode == 'tracking_by_detection':
            kft = KalmanFilterTracker(bboxes, confs, names)
        initialize = False
    frame = face_recognizer.annotate(frame, bboxes, names=names)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(0) == ord('q'):
        break
    writer.write(frame)
cv2.destroyAllWindows()
cap.release()
writer.release()